In [2]:
df = spark.read.option("header", True).option("inferSchema", True).csv('s3a://yc-dataproc-tasks/data/transaction_data.csv')

Waiting for an Apache Livy session to start...
Apache Livy session has started.



In [ ]:
#   ===============    Обзор полученных данных   ===============

In [11]:
# Показываем 10 строк в обрезанном состоянии
df.show(10)

+------+-------------+--------------------+--------+--------------------+----------------------+-----------+--------------+
|UserId|TransactionId|     TransactionTime|ItemCode|     ItemDescription|NumberOfItemsPurchased|CostPerItem|       Country|
+------+-------------+--------------------+--------+--------------------+----------------------+-----------+--------------+
|278166|      6355745|Sat Feb 02 12:50:...|  465549|FAMILY ALBUM WHIT...|                     6|      11.73|United Kingdom|
|337701|      6283376|Wed Dec 26 09:06:...|  482370|LONDON BUS COFFEE...|                     3|       3.52|United Kingdom|
|267099|      6385599|Fri Feb 15 09:45:...|  490728|SET 12 COLOUR PEN...|                    72|        0.9|        France|
|380478|      6044973|Fri Jun 22 07:14:...|  459186|UNION JACK FLAG L...|                     3|       1.73|United Kingdom|
|    -1|      6143225|Mon Sep 10 11:58:...| 1733592| WASHROOM METAL SIGN|                     3|        3.4|United Kingdom|
|285957|

In [12]:
# Выбирает из датафрейма n верхних строк
df.limit(5).show(truncate = False)

+------+-------------+----------------------------+--------+---------------------------------+----------------------+-----------+--------------+
|UserId|TransactionId|TransactionTime             |ItemCode|ItemDescription                  |NumberOfItemsPurchased|CostPerItem|Country       |
+------+-------------+----------------------------+--------+---------------------------------+----------------------+-----------+--------------+
|278166|6355745      |Sat Feb 02 12:50:00 IST 2019|465549  |FAMILY ALBUM WHITE PICTURE FRAME |6                     |11.73      |United Kingdom|
|337701|6283376      |Wed Dec 26 09:06:00 IST 2018|482370  |LONDON BUS COFFEE MUG            |3                     |3.52       |United Kingdom|
|267099|6385599      |Fri Feb 15 09:45:00 IST 2019|490728  |SET 12 COLOUR PENCILS DOLLY GIRL |72                    |0.9        |France        |
|380478|6044973      |Fri Jun 22 07:14:00 IST 2018|459186  |UNION JACK FLAG LUGGAGE TAG      |3                     |1.73       |U

In [9]:
import pprint

# Получаем массив n строк
pp = pprint.PrettyPrinter()
pp.pprint(df.take(2))

[Row(UserId=278166, TransactionId=6355745, TransactionTime='Sat Feb 02 12:50:00 IST 2019', ItemCode=465549, ItemDescription='FAMILY ALBUM WHITE PICTURE FRAME', NumberOfItemsPurchased=6, CostPerItem=11.73, Country='United Kingdom'),
 Row(UserId=337701, TransactionId=6283376, TransactionTime='Wed Dec 26 09:06:00 IST 2018', ItemCode=482370, ItemDescription='LONDON BUS COFFEE MUG', NumberOfItemsPurchased=3, CostPerItem=3.52, Country='United Kingdom')]
[Row(UserId=278166, TransactionId=6355745, TransactionTime='Sat Feb 02 12:50:00 IST 2019', ItemCode=465549, ItemDescription='FAMILY ALBUM WHITE PICTURE FRAME', NumberOfItemsPurchased=6, CostPerItem=11.73, Country='United Kingdom'), Row(UserId=337701, TransactionId=6283376, TransactionTime='Wed Dec 26 09:06:00 IST 2018', ItemCode=482370, ItemDescription='LONDON BUS COFFEE MUG', NumberOfItemsPurchased=3, CostPerItem=3.52, Country='United Kingdom')]


In [14]:
import pprint

# Показываем 3 строки в обрезанном состоянии
pp = pprint.PrettyPrinter()
pp.pprint(df.limit(3).collect())

[Row(UserId=278166, TransactionId=6355745, TransactionTime='Sat Feb 02 12:50:00 IST 2019', ItemCode=465549, ItemDescription='FAMILY ALBUM WHITE PICTURE FRAME', NumberOfItemsPurchased=6, CostPerItem=11.73, Country='United Kingdom'),
 Row(UserId=337701, TransactionId=6283376, TransactionTime='Wed Dec 26 09:06:00 IST 2018', ItemCode=482370, ItemDescription='LONDON BUS COFFEE MUG', NumberOfItemsPurchased=3, CostPerItem=3.52, Country='United Kingdom'),
 Row(UserId=267099, TransactionId=6385599, TransactionTime='Fri Feb 15 09:45:00 IST 2019', ItemCode=490728, ItemDescription='SET 12 COLOUR PENCILS DOLLY GIRL ', NumberOfItemsPurchased=72, CostPerItem=0.9, Country='France')]


In [ ]:
#   ===============    Изучение структуры данных   ===============

In [16]:
import pprint

# Выбирает из датафрейма n верхних строк
pp = pprint.PrettyPrinter()
pp.pprint(df.columns)

['UserId',
 'TransactionId',
 'TransactionTime',
 'ItemCode',
 'ItemDescription',
 'NumberOfItemsPurchased',
 'CostPerItem',
 'Country']


In [24]:
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType
import pprint

pp = pprint.PrettyPrinter()
print("Выводим список атрибутов с помощью метода dtypes")
pp.pprint(df.dtypes)
print("\nВыводим список атрибутов с помощью метода schema")
pp.pprint(df.schema)

df_fields = df.schema.fields

print("\nПреобразовываем все атрибуты с типом IntegerType в DoubleType")
new_schema = StructType([StructField(field.name, DoubleType(), field.nullable) if field.dataType == IntegerType() else field for field in df_fields])

pp.pprint(new_schema.fields)

Выводим список атрибутов с помощью метода dtypes
[('UserId', 'int'),
 ('TransactionId', 'int'),
 ('TransactionTime', 'string'),
 ('ItemCode', 'int'),
 ('ItemDescription', 'string'),
 ('NumberOfItemsPurchased', 'int'),
 ('CostPerItem', 'double'),
 ('Country', 'string')]

Выводим список атрибутов с помощью метода schema
StructType(List(StructField(UserId,IntegerType,true),StructField(TransactionId,IntegerType,true),StructField(TransactionTime,StringType,true),StructField(ItemCode,IntegerType,true),StructField(ItemDescription,StringType,true),StructField(NumberOfItemsPurchased,IntegerType,true),StructField(CostPerItem,DoubleType,true),StructField(Country,StringType,true)))

Преобразовываем все атрибуты с типом IntegerType в DoubleType
[StructField(UserId,DoubleType,true),
 StructField(TransactionId,DoubleType,true),
 StructField(TransactionTime,StringType,true),
 StructField(ItemCode,DoubleType,true),
 StructField(ItemDescription,StringType,true),
 StructField(NumberOfItemsPurchased,Doubl

In [ ]:
df.printSchema()

In [25]:
df = spark\
        .read\
        .option("inferSchema", True)\
        .option("header", True)\
        .csv('s3a://yc-dataproc-tasks/data/transaction_data.csv')

# Выбирает из датафрейма n верхних строк
df.printSchema()

root
 |-- UserId: integer (nullable = true)
 |-- TransactionId: integer (nullable = true)
 |-- TransactionTime: string (nullable = true)
 |-- ItemCode: integer (nullable = true)
 |-- ItemDescription: string (nullable = true)
 |-- NumberOfItemsPurchased: integer (nullable = true)
 |-- CostPerItem: double (nullable = true)
 |-- Country: string (nullable = true)


In [ ]:
from pyspark.sql.functions import col,lit
from datetime import datetime

current_datetime = datetime.now()

df.withColumn('processed_dttm', lit(current_datetime)).show(5, False)

In [ ]:
df.select("UserId", "Country").show(5, False)

In [ ]:
df.select("Country").distinct().show()

In [ ]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

schema = StructType([ \
      StructField("UserId", IntegerType()), \
      StructField("TransactionId", IntegerType()), \
      StructField("TransactionTime", StringType()), \
      StructField("ItemCode", IntegerType()), \
      StructField("ItemDescription", StringType()), \
      StructField("NumberOfItemsPurchased", StringType()), \
      StructField("CostPerItem", StringType()), \
      StructField("Country", StringType()) \
    ])
        
df = spark\
        .read\
        .option("header", True)\
        .schema(schema)\
        .csv('s3a://yc-dataproc-tasks/data/transaction_data.csv')
    
df.printSchema()

In [ ]:
#   ===============    Манипуляция со столбцами   ===============

In [ ]:
from pyspark.sql.functions import col, lit

# Выбор столбцов
print("Передаем на вход массив атрибутов")
df.select(df.columns).show(2, False)

print("Атрибуты по именам")
df.select("UserId", "Country").show(2, False)

print("Используем объект Column")
df.select((col("UserId") * 100).alias("UserId"), col("Country"), lit("hello World").alias("greetings")).show(2, False)

print("Обращение к атрибуту")
df.select(df.ItemCode, (df.TransactionId - 1000000).alias("newTransaction")).show(2, False)

In [ ]:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

# Выбор столбцов
df\
    .dropDuplicates(["Country"])\
    .selectExpr("UserId",\
                "date_format(to_timestamp(TransactionTime, 'E MMM dd HH:mm:ss z yyyy'), 'HH:mm:ss.SSS yyyy/MMM/dd') as timestamp",\
                "concat(Country, ' test') as cntr",\
                "'hello world!' as greeting"
               )\
    .show(5, False)

In [ ]:
# Добавление и изменение атрибутов
from pyspark.sql.functions import when, lower, regexp_replace, length, expr

df\
    .select("ItemCode", "Country")\
    .dropDuplicates(["Country"])\
    .withColumn("spaceCount", when(length(regexp_replace(col("Country"), "\S", "")) == 1, "Single Space")
                .when(length(regexp_replace(col("Country"), "\S", "")) > 1, "Multiple Space")
                .otherwise("No spaces")
               )\
    .withColumn("ItemCode", expr("case when ItemCode > 450000 then ItemCode + 500000 else round(sqrt(ItemCode), 2) end"))\
    .show(30, truncate = False)


In [ ]:
#   ===============    Манипуляция со строками датафрейма   ===============

In [ ]:
# Сортировка
from pyspark.sql.functions import col

df_without_duplicates = df.select("UserId", "TransactionId", "ItemCode").dropDuplicates(["UserId"])

df_without_duplicates.sort(col("UserId")).show(10)
print("Сортировка с использованием sort")
df_without_duplicates.sort(col("UserId").desc(), col("ItemCode").asc()).show(10)
print("Сортировка с использованием orderBy")
df_without_duplicates.orderBy(col("UserId").desc(), col("ItemCode").asc()).show(10)

In [ ]:
# Фильтрация
from pyspark.sql.functions import col

item_df = df.select("ItemCode", "ItemDescription", "CostPerItem")

print("Фильтрация с использованием where")
item_df.where((col("CostPerItem") > 2.0) & (col("CostPerItem") < 3.0)).show(truncate = False)
print("Фильтрация с использованием filter")
item_df.filter((col("CostPerItem") > 2.0) & (col("ItemDescription").like("LUNCH%"))).show()

In [ ]:
# Получение уникальных строк
user_df = df.select("UserId", "TransactionId", "Country")

print("Убираем полные дубли (по всем строкам)")
user_df.distinct().sort("UserId", "Country").show(10)
print("Убираем дубли только для атрибута UserId")
user_df.dropDuplicates(["UserId"]).sort("UserId").show(10)
print("Убираем дубли для атрибутов UserId и Country")
user_df.dropDuplicates(["UserId", "Country"]).sort("UserId").show(10)

In [ ]:
#   ===============    Агрегаты и обогащение   ===============

In [ ]:
from pyspark.sql.functions import avg, count, min, max, sum

# Получаем среднюю стоимость для каждого товара и количество купленных товаров
print("Получаем среднюю стоимость товара и сумму, сколько раз товар был куплен")
df.select("ItemCode", "CostPerItem", "NumberOfItemsPurchased")\
    .groupBy("ItemCode")\
    .agg(avg("CostPerItem").alias("AverageCost"), sum("NumberOfItemsPurchased").alias("CountOfPurchasedItems"))\
    .show(10, False)

# Получаем количество товаров, которое приобрел каждый пользователь
print("Получаем данные о том, сколько товаров приобрел каждый пользователь")
df.select("UserId", "ItemCode").distinct().groupBy("UserId").agg(count("*").alias("ItemCount")).show(10, False)

In [ ]:
from pyspark.sql.functions import col, lit
from datetime import datetime

# Определяем текущую временную метку
current_datetime = datetime.now()

# Обогащаем датафрейм новым полем - информация о времени обработки датафрейма
df.select("UserId", "TransactionId", "ItemCode").withColumn('processed_dttm', lit(current_datetime)).show(5, False)

In [ ]:
#   ===============    Функции   ===============

In [ ]:
from pyspark.sql.functions import col, lower, upper, substring, split, trim, regexp_replace, length

# Функции для работы со строками
df\
    .dropDuplicates(["Country"])\
    .limit(10)\
    .select(
        "Country",\
        lower("Country").alias("lower"),\
        upper("Country").alias("upper"),\
        substring("Country", 1,3).alias("substring"),\
        split("Country", " ").alias("split"),
        trim("Country").alias("trim"),\
        regexp_replace("Country", "e", "EE").alias("regexpl_replace"),\
        length("Country").alias("length")\
    ).show()

In [ ]:
from pyspark.sql.functions import abs, round, ceil, pow, sqrt

# Функции для работы с числами

df\
    .dropDuplicates(["CostPerItem"])\
    .limit(15)\
    .select(\
        "CostPerItem",\
        abs("CostPerItem").alias("abs"),\
        ceil("CostPerItem").alias("ceil"),\
        round(pow("CostPerItem", 2), 3).alias("powWithRound"),\
        sqrt(abs("CostPerItem")).alias("sqrt")\
    ).sort("CostPerItem").show()

In [ ]:
from pyspark.sql.functions import col, current_date, current_timestamp, dayofmonth, date_add, date_format, to_timestamp, trunc, month, year

spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

# Функции для работы с датой и временем
df\
    .select(\
            col("TransactionTime"),\
            current_date().alias("CurrentDate"),\
            current_timestamp().alias("CurrentTimestamp"),\
            trunc(current_timestamp(), "month").alias("Trunc"),\
            to_timestamp(col("TransactionTime"), "E MMM dd HH:mm:ss z yyyy").alias("TransformedTranscation"),\
            date_format(date_add(current_timestamp(), 30), "dd/MM/yyyy").alias("DateAdd")\
           )\
    .withColumn("year", year("TransformedTranscation"))\
    .withColumn("month", month("TransformedTranscation"))\
    .withColumn("day", dayofmonth("TransformedTranscation"))\
    .show(10, truncate = False)

In [ ]:
from pyspark.sql.functions import array_contains, array_join, array_sort, col, size, split

# Функции для работы с массивами
df\
    .select(col("Country"), split(col("Country"), " ").alias("splittedCountry"))\
    .distinct()\
    .sort(col("Country").desc())\
    .select(col("Country"),\
            col("splittedCountry"),\
            array_contains(col("splittedCountry"), "United").alias("contains"),\
            size(col("splittedCountry")).alias("size"),\
            array_join(array_sort(col("splittedCountry")), "_").alias("join")\
           )\
    .show(20, False)